In [1]:
import numpy as np
import pandas as pd

np.random.seed(333)

df = pd.read_csv('./data/merged_probs_odds.csv') # labels: "home wins" -> 1; "draw": 0; "away" -> -1
df.tail()
df = df.sort_values("date_x", ascending = True)

In [2]:
from tqdm import tqdm

def make_trajectories(df, trajectory_len=30, keep_cols=['spi1', 'spi2', 'prob_home', 'prob_tie', 'prob_away', 'odds_home', 'odds_draw', 'odds_away',
                                                        'proj_home_score', 'proj_away_score', 'home_goals', 'away_goals', 'label']):
    num_traj = int(df.shape[0] // trajectory_len)
    trajectories = []
    for i in tqdm(range(num_traj)):
        df = df.sample(frac=1.0) # shuffle
        traj = df.iloc[:trajectory_len] # keep first {traj_len} random games
        traj = traj.sort_values(by='date_x', ascending=True) # make game sample chronologic
        trajectories.append(traj)
    return trajectories

In [3]:
rename_dict = {"odds_home": "home", "odds_draw": "draw", "odds_away": "away", "label": "result"}

odds_df = df[["date_x", "home_team", "away_team", "odds_home", "odds_draw", "odds_away", "label"]].rename(rename_dict, axis = 1)
odds_df['result'] = odds_df['result'].map({1: 0, -1: 2, 0: 1})

In [4]:
traj_list = make_trajectories(odds_df, keep_cols = ["home_team", "away_team", "odds_home", "odds_draw", "odds_away", "label"])

100%|██████████| 358/358 [00:00<00:00, 980.24it/s] 


In [5]:
train_list = traj_list[:300]
test_list = traj_list[300:]

train_df = pd.concat(train_list)
test_df = pd.concat(test_list)

train_df

,date_x,home_team,away_team,home,draw,away,result
544,2017-08-09,Ponte Preta,Fluminense,2.17,3.27,3.29,1
812,2017-09-16,Guingamp,Lille,2.15,3.30,3.50,0
970,2017-09-30,Udinese,Sampdoria,2.70,3.25,2.60,0
2073,2018-08-15,Vissel Kobe,Sanfrecce Hiroshima,2.42,3.40,2.78,1
2271,2018-09-02,Albacete,Cordoba,1.95,3.10,4.33,0
...,...,...,...,...,...,...,...
8884,2022-06-24,Internacional,Coritiba,1.76,3.57,4.70,0
9052,2022-08-03,Portland Timbers,Nashville SC,2.22,3.56,3.03,1
9725,2022-10-01,Ajax,Go Ahead Eagles,1.08,11.29,25.19,1
9971,2022-10-16,Real Madrid,Barcelona,2.28,3.64,3.05,0


In [6]:
from oddsgym.envs.soccer import ThreeWaySoccerOddsEnv
from oddsgym.envs.base_percentage import BasePercentageOddsEnv

Trial #1: Bet fixed amounts alway on a home win

Sources: 

https://colab.research.google.com/drive/1eqODB6TYWUaPq-SG1LD8kVcw1VMqEUyU#scrollTo=juMw2p58Jv8r

In [7]:
#Test model and append total ending rewards 
eps_length = test_list[0].shape[0]
num_traj = len(test_list)
rewards = []

for i in range(num_traj):
    env = ThreeWaySoccerOddsEnv(test_list[i], starting_bank = 1000)
    env.reset()
    for l in range(eps_length):
        #print(env.render())
        obs, reward, done, info = env.step(1)
        if done:
            rewards.append(reward+1000)
            break

print(sum(rewards)/len(rewards))
print(max(rewards))

999.9086206896551
1006.17


Trial #2: Bet fixed amounts always randomly

In [8]:
#Test model and append total ending rewards 
eps_length = test_list[0].shape[0]
num_traj = len(test_list)
rewards = []

for i in range(num_traj):
    env = ThreeWaySoccerOddsEnv(test_list[i])
    env.reset()
    for l in range(eps_length):
        #print(env.render())
        obs, reward, done, info = env.step(env.action_space.sample())
        if done:
            rewards.append(reward+1000)
            break

print(sum(rewards)/len(rewards))
print(max(rewards))

999.6655172413792
1006.17


Trial #3: Bet percentage amounts always randomly

In [9]:
#Test model and append total ending rewards 
eps_length = test_list[0].shape[0]
num_traj = len(test_list)
rewards = []

for i in range(num_traj):
    odds_df = test_list[i][["home", "draw",	"away"]].to_numpy()
    results_df = test_list[i]["result"].tolist()
    env = BasePercentageOddsEnv(odds_df, ["home", "draw", "away"], results_df)
    env.reset()
    for l in range(eps_length):
        #print(env.render())
        obs, reward, done, info = env.step(env.action_space.sample())
        if done:
            rewards.append(reward+1000)
            break

print(sum(rewards)/len(rewards))
print(max(rewards))

/Users/bryanchia/opt/anaconda3/envs/cs224r_project/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


985.6115625288363
1000


Trial #4: Fixed Baselines, train policy using PPO2

Sources:

https://colab.research.google.com/drive/1eqODB6TYWUaPq-SG1LD8kVcw1VMqEUyU#scrollTo=juMw2p58Jv8r 

https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/3_multiprocessing.ipynb#scrollTo=AvO5BGrVv2Rk 

https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html

In [10]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines import PPO2
from stable_baselines.common import set_global_seeds

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/Users/bryanchia/opt/anaconda3/envs/cs224r_project/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [11]:
def make_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = ThreeWaySoccerOddsEnv(train_list[env_id])
        # Important: use a different seed for each environment
        env.seed(seed + rank)
        return env
    set_global_seeds(seed)
    return _init

In [12]:
#Train model and append total ending rewards 
eps_length = train_list[0].shape[0]
num_traj = len(train_list)
rewards = []

learning_env = DummyVecEnv([make_env(i, i) for i in range(num_traj)])
model = PPO2(MlpPolicy, learning_env, verbose=1)
obs = learning_env.reset()
print(model.action_probability(obs))
model.learn(total_timesteps=3000)






Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



[[0.12498855 0.12528048 0.12514114 ... 0.12551267 0.1246426  0.12451547]
 [0.1250485  0.12558624 0.12505959 ... 0.12475454 0.12494054 0.12421407]
 [0.12502126 0.12515979 0.12527171 ... 0.12575656 0.12454916 0.12460402]
 ...
 [0.12574993 0.12469894 0.12595485 ... 0.12626122 0.12430038 0.12453921]
 [0.12537202 0.12488899 0.12575236 ... 0.12624873 0.12433683 0.12466716]
 [0.12502609 0.12513088 0.12531522 ... 0.12582915 0.12451967 0.12462526]]


In [13]:
#Test model and append total ending rewards 
eps_length = test_list[0].shape[0]
num_traj = len(test_list)
rewards = []

for i in range(num_traj):
    env = ThreeWaySoccerOddsEnv(test_list[i])
    obs = env.reset()
    for l in range(eps_length):
        #print(env.render())
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        if done:
            rewards.append(reward+1000)
            break

print(sum(rewards)/len(rewards))
print(max(rewards))

999.9546551724139
1002.96


Trial #5: Percentage Baselines, train policy using PPO2

In [14]:
def make_perc_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        odds_df = train_list[env_id][["home", "draw",	"away"]].to_numpy()
        results_df = train_list[env_id]["result"].tolist()
        env = BasePercentageOddsEnv(odds_df, ["home", "draw", "away"], results_df)
        # Important: use a different seed for each environment
        env.seed(seed + rank)
        return env
    set_global_seeds(seed)
    return _init

In [17]:
#Train model and append total ending rewards 
eps_length = train_list[0].shape[0]
num_traj = len(train_list)
rewards = []

learning_env = DummyVecEnv([make_perc_env(i, i) for i in range(num_traj)])
model = PPO2(MlpPolicy, learning_env, verbose=1)
obs = learning_env.reset()
print(model.action_probability(obs))
model.learn(total_timesteps=3000)

[array([[-0.0050107 , -0.00712751, -0.0076907 ],
       [-0.0046306 , -0.00900902, -0.00859055],
       [-0.00545115, -0.00636952, -0.00727941],
       [-0.00478687, -0.00669155, -0.00756788],
       [-0.00582008, -0.00726601, -0.00741429],
       [-0.00515582, -0.00696524, -0.00758632],
       [-0.00539629, -0.00548125, -0.00708503],
       [-0.00491795, -0.00882078, -0.00843984],
       [-0.00564642, -0.00555529, -0.00697393],
       [-0.00497422, -0.00515383, -0.00699646],
       [-0.00510776, -0.00682327, -0.00760084],
       [-0.00546265, -0.00717006, -0.0075948 ],
       [-0.00613437, -0.01005522, -0.00813334],
       [-0.00518323, -0.00381718, -0.00772042],
       [-0.00521253, -0.00584823, -0.00714031],
       [-0.00499432, -0.00889319, -0.00846201],
       [-0.00514315, -0.00882761, -0.00838541],
       [-0.00516314, -0.00407965, -0.00734948],
       [-0.00547132, -0.00788493, -0.00787292],
       [-0.00552235, -0.00863519, -0.00814348],
       [-0.00525907, -0.00566968, -0.00

In [18]:
#Test model and append total ending rewards 
eps_length = test_list[0].shape[0]
num_traj = len(test_list)
rewards = []

for i in range(num_traj):
    odds_df = test_list[i][["home", "draw",	"away"]].to_numpy()
    results_df = test_list[i]["result"].tolist()
    env = BasePercentageOddsEnv(odds_df, ["home", "draw", "away"], results_df)
    env.reset()
    for l in range(eps_length):
        #print(env.render())
        obs, reward, done, info = env.step(env.action_space.sample())
        if done:
            rewards.append(reward+1000)
            break

print(sum(rewards)/len(rewards))
print(max(rewards))

994.1746530575484
1005.3678946113587
